In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

train_data = pd.read_csv('dataset/price_data_tr.csv')
heldout_data = pd.read_csv('dataset/price_data_val.csv')
test_data = pd.read_csv('dataset/price_data_ts.csv')

In [3]:
print(train_data.shape)
print(heldout_data.shape)
print(test_data.shape)

(12968, 21)
(4323, 21)
(4322, 21)


In [4]:
# data.columns
# 'id', 'date', 
# 'price', 
# 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
# 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15'

In [5]:
print(train_data.iloc[0:2, [0,1,2,3]])
print(heldout_data.iloc[0:2, [0,1,2,3]])
print(test_data.iloc[0:2, [0,1,2,3]])

           id             date     price  bedrooms
0  1310440590  20150413T000000  440000.0         3
1  1437580480  20140918T000000  994000.0         5
           id             date     price  bedrooms
0  7229800175  20140604T000000  453500.0         5
1  3905080870  20150206T000000  510000.0         3
           id             date  price  bedrooms
0  7000100850  20140926T000000    NaN         4
1  4037400280  20140923T000000    NaN         3


In [15]:
corr = train_data.corr(method='pearson')
corr.iloc[:, 1]

id              -0.013029
price            1.000000
bedrooms         0.322938
bathrooms        0.525247
sqft_living      0.697376
sqft_lot         0.080147
floors           0.254591
waterfront       0.277248
view             0.393101
condition        0.032984
grade            0.666064
sqft_above       0.603758
sqft_basement    0.320450
yr_built         0.055129
yr_renovated     0.110779
zipcode         -0.049526
lat              0.306516
long             0.017529
sqft_living15    0.587954
sqft_lot15       0.076897
Name: price, dtype: float64

In [29]:
yr_sum = train_data.yr_built + train_data.yr_renovated
yr_sum.apply(lambda x : x / 2 if x > 3000 else x)

0        1997.0
1        2005.0
2        1956.0
3        1983.0
4        1952.0
          ...  
12963    2007.0
12964    1998.0
12965    1944.0
12966    1977.0
12967    1975.0
Length: 12968, dtype: float64

In [33]:
yr_sum

0        1997
1        2005
2        1956
3        3966
4        1952
         ... 
12963    2007
12964    1998
12965    1944
12966    1977
12967    1975
Length: 12968, dtype: int64

In [32]:
train_data = train_data.drop(['yr_built','yr_renovated'], axis=1)
train_data.shape

(12968, 19)

In [34]:
train_data['yr'] = yr_sum.apply(lambda x : x / 2 if x > 3000 else x)

In [35]:
train_data

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,zipcode,lat,long,sqft_living15,sqft_lot15,yr
0,1310440590,20150413T000000,440000.0,3,2.50,2290,6302,2.0,0,0,3,9,2290,0,98058,47.4350,-122.107,2700,7500,1997.0
1,1437580480,20140918T000000,994000.0,5,3.25,4260,7861,2.0,0,0,3,10,4260,0,98074,47.6110,-121.992,4020,7528,2005.0
2,7617500075,20140721T000000,427000.0,3,1.75,2000,7111,1.0,0,0,4,7,1360,640,98177,47.7676,-122.373,1830,9000,1956.0
3,8122100130,20140618T000000,415000.0,3,1.75,1270,4800,1.0,0,0,3,7,1270,0,98126,47.5362,-122.376,1220,4800,1983.0
4,4476400275,20150114T000000,335000.0,3,1.75,2100,8298,1.0,0,0,4,7,1230,870,98166,47.4601,-122.360,1700,10830,1952.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12963,1773100604,20140721T000000,346000.0,3,3.25,1500,1442,2.0,0,0,3,8,1150,350,98106,47.5592,-122.362,1500,1533,2007.0
12964,6403510090,20141111T000000,437500.0,4,2.50,2680,7513,2.0,0,0,3,8,2680,0,98059,47.4956,-122.161,2640,7243,1998.0
12965,2124049160,20150416T000000,440000.0,6,3.00,2510,5310,1.0,0,0,4,7,1460,1050,98108,47.5533,-122.304,1390,5407,1944.0
12966,1310960190,20141021T000000,263000.0,3,1.75,1660,7210,1.0,0,0,4,7,1660,0,98032,47.3609,-122.274,2150,7350,1977.0


In [36]:
corr = train_data.corr(method='pearson')
corr.iloc[:, 1]

id              -0.013029
price            1.000000
bedrooms         0.322938
bathrooms        0.525247
sqft_living      0.697376
sqft_lot         0.080147
floors           0.254591
waterfront       0.277248
view             0.393101
condition        0.032984
grade            0.666064
sqft_above       0.603758
sqft_basement    0.320450
zipcode         -0.049526
lat              0.306516
long             0.017529
sqft_living15    0.587954
sqft_lot15       0.076897
yr               0.079670
Name: price, dtype: float64